In [119]:
pip install Faker

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sqlite3
from faker import Faker
import os
import random
import pandas as pd
from datetime import datetime, timedelta


fake = Faker()
faker = Faker('es_ES')
data = []


: 

In [109]:
# Configuración de generación
n_personas = 40000
dias_laborales = pd.date_range(start="2023-03-01", end="2025-01-01", freq="B")  # solo días hábiles

# Datos base
nombres = ["PEDRO", "LUCIA", "MARTA", "JUAN", "CARLOS", "ANDREA", "SOFIA", "RAUL", "JORGE", "MARIA"]
apellidos = ["PASCAL", "RAMIREZ", "RODRIGUEZ", "GOMEZ", "DIAZ", "FERNANDEZ", "TORRES", "MARTINEZ", "LOPEZ", "NAVARRO"]
departamentos = {
    "IT": 2,
    "FINANZAS": 5,
    "RRHH": 3,
    "LEGAL": 4,
    "LOGISTICA": 1,
    "OPERACIONES": 6
}
tipo_personal = ["INTERNO", "EXTERNO"]

# Generar registros
registros = []
for i in range(n_personas):
    id_persona = random.randint(10000, 99999)
    nombre = random.choice(nombres)
    apellido = random.choice(apellidos)
    tipo = random.choice(tipo_personal)
    dpto = random.choice(list(departamentos.keys()))
    piso = departamentos[dpto]
    dias_trabajo = random.sample(list(dias_laborales), k=random.randint(2, 6))
    
    for dia in dias_trabajo:
        registros.append([
            id_persona, nombre, apellido, tipo, dpto, piso, dia.date()
        ])

# Crear DataFrame
df_personal = pd.DataFrame(registros, columns=[
    "ID", "NOMBRE", "APELLIDO", "PERSONAL", "DEPARTAMENTO", "PISO_TRABAJO", "DIA_TRABAJO"
])


In [110]:
df_personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159798 entries, 0 to 159797
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ID            159798 non-null  int64 
 1   NOMBRE        159798 non-null  object
 2   APELLIDO      159798 non-null  object
 3   PERSONAL      159798 non-null  object
 4   DEPARTAMENTO  159798 non-null  object
 5   PISO_TRABAJO  159798 non-null  int64 
 6   DIA_TRABAJO   159798 non-null  object
dtypes: int64(2), object(5)
memory usage: 8.5+ MB


In [111]:
df_personal

,ID,NOMBRE,APELLIDO,PERSONAL,DEPARTAMENTO,PISO_TRABAJO,DIA_TRABAJO
0,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-07-27
1,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-10-04
2,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-07-19
3,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-11-06
4,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2024-04-29
...,...,...,...,...,...,...,...
159793,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2024-12-20
159794,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2023-11-03
159795,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2024-10-24
159796,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2023-05-15


In [90]:
import sqlite3

# Crear o conectar a la base de datos SQLite (se crea archivo si no existe)
conn = sqlite3.connect("ocupacion_personal.db")

# Guardar el DataFrame en la tabla 'datos_personal'
df_personal.to_sql("datos_personal", conn, if_exists="replace", index=False)

# Verificar que se creó correctamente (opcional)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM datos_personal")
print("Filas insertadas:", cursor.fetchone()[0])

conn.close()


Filas insertadas: 180709


In [91]:
ruta_excel = r"C:\Users\segur\Desktop\Data Science SL - The Bridge Repositorios\ONLINE_DS_THEBRIDGE_SL\Proyecto_EDA\SCR\SCR DATA\datos_ocupacion\occupancy_report_full.xlsx"
df_occupancy = pd.read_excel(ruta_excel)

In [92]:
import sqlite3

conn = sqlite3.connect("ocupacion_personal.db")
df_occupancy.to_sql("occupancy_report_full", conn, if_exists="replace", index=False)
conn.close()

In [112]:
# Supongamos que ya cargaste tu archivo Excel a df_occupancy
def generar_hora(tipo):
    if tipo.upper() == "INTERNO":
        return f"{random.randint(7, 10)}:{random.choice(['00', '15', '30', '45'])}"
    else:
        return f"{random.randint(9, 14)}:{random.choice(['00', '15', '30', '45'])}"

df_personal["HORA_ACCESO"] = df_occupancy["TIPO_PERSONAL"].apply(generar_hora)

In [118]:
df_personal

,ID,NOMBRE,APELLIDO,PERSONAL,DEPARTAMENTO,PISO_TRABAJO,DIA_TRABAJO,HORA_ACCESO
0,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-07-27,14:45
1,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-10-04,8:15
2,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-07-19,14:15
3,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2023-11-06,10:45
4,45391,JORGE,TORRES,EXTERNO,FINANZAS,5,2024-04-29,14:15
...,...,...,...,...,...,...,...,...
159793,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2024-12-20,NaN
159794,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2023-11-03,NaN
159795,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2024-10-24,NaN
159796,17190,JORGE,NAVARRO,EXTERNO,OPERACIONES,6,2023-05-15,NaN


In [105]:
df_occupancy["DIA"] = pd.to_datetime("1899-12-30") + pd.to_timedelta(df_occupancy["DIA"], unit="D")

TypeError: dtype datetime64[ns] cannot be converted to timedelta64[ns]

In [102]:
print(df_occupancy["DIA"].head())


0   2023-03-01
1   2023-03-01
2   2023-03-02
3   2023-03-02
4   2023-03-03
Name: DIA, dtype: datetime64[ns]


In [103]:
print(df_occupancy.dtypes)

AÑO                       int64
MES                       int64
TIPO_PERSONAL            object
DIA              datetime64[ns]
OCUPACION                 int64
DIA SEMANA               object
HORA ACCESO             float64
HORA_ACCESO              object
dtype: object


In [104]:
df_occupancy

,AÑO,MES,TIPO_PERSONAL,DIA,OCUPACION,DIA SEMANA,HORA ACCESO,HORA_ACCESO
0,2023,3,EXTERNO,2023-03-01,72,miércoles,NaN,14:45
1,2023,3,INTERNO,2023-03-01,392,miércoles,NaN,9:45
2,2023,3,EXTERNO,2023-03-02,79,jueves,NaN,11:15
3,2023,3,INTERNO,2023-03-02,409,jueves,NaN,7:30
4,2023,3,EXTERNO,2023-03-03,71,viernes,NaN,12:45
...,...,...,...,...,...,...,...,...
1215,2025,1,INTERNO,2025-01-29,426,miércoles,NaN,8:00
1216,2025,1,EXTERNO,2025-01-30,71,jueves,NaN,10:15
1217,2025,1,INTERNO,2025-01-30,594,jueves,NaN,7:15
1218,2025,1,EXTERNO,2025-01-31,64,viernes,NaN,13:30


In [107]:
df_occupancy.dropna()

,AÑO,MES,TIPO_PERSONAL,DIA,OCUPACION,DIA SEMANA,HORA ACCESO,HORA_ACCESO


In [108]:
df_occupancy

,AÑO,MES,TIPO_PERSONAL,DIA,OCUPACION,DIA SEMANA,HORA ACCESO,HORA_ACCESO
0,2023,3,EXTERNO,2023-03-01,72,miércoles,NaN,14:45
1,2023,3,INTERNO,2023-03-01,392,miércoles,NaN,9:45
2,2023,3,EXTERNO,2023-03-02,79,jueves,NaN,11:15
3,2023,3,INTERNO,2023-03-02,409,jueves,NaN,7:30
4,2023,3,EXTERNO,2023-03-03,71,viernes,NaN,12:45
...,...,...,...,...,...,...,...,...
1215,2025,1,INTERNO,2025-01-29,426,miércoles,NaN,8:00
1216,2025,1,EXTERNO,2025-01-30,71,jueves,NaN,10:15
1217,2025,1,INTERNO,2025-01-30,594,jueves,NaN,7:15
1218,2025,1,EXTERNO,2025-01-31,64,viernes,NaN,13:30


In [ ]:
    cursor.execute("DROP TABLE IF EXISTS operaciones")

 
    cursor.execute("""
    CREATE TABLE operaciones (
        id_operacion INTEGER PRIMARY KEY,
        id_proveedor INTEGER,
        id_pieza INTEGER,
        fecha TEXT,
        id_factura INTEGER
    )
    """)

In [ ]:
df_operaciones.to_sql('operaciones', cnx, if_exists='append', index=False)
print("\nDatos cargados correctamente en bdd_new.db.")

In [ ]:
#PRUEBA DE QUE FUNCIONA 

query = ''' 
SELECT *
FROM operaciones
WHERE id_factura > 5010
'''
pd.read_sql(query, cnx)

In [ ]:
cnx.close() #CIERRE CONEXTION

In [ ]:
with sqlite3.connect("bdd_testing.db") as cnx: #CHECK BDD
    cursor = cnx.cursor()